# Workflow Automation with Python SDK

Welcome to this interactive notebook!  
Here, you'll learn how to **create**, **configure**, and **manage workflows** using the Python SDK.

---

## 📋 What You'll Learn

- How to set up your environment
- Step-by-step workflow creation
- Managing and executing workflow tasks programmatically

---

> **Tip:** Use the code cells to experiment and try out each step as you follow along.

Let's get started 🚀

In [2]:
# ModelCub SDK Playground
# Testing SDK operations with logging and metadata

# ============================================================
# Setup: Enable Logging
# ============================================================

import logging
import sys

# Configure logging to see all service operations
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

# Get the ModelCub logger
logger = logging.getLogger("modelcub")
logger.setLevel(logging.DEBUG)

print("✅ Logging enabled\n")

# ============================================================
# 1. Project Operations
# ============================================================

from modelcub import Project
from pathlib import Path
import shutil

# Clean up if exists
test_project_path = Path("./test_sdk_project")
if test_project_path.exists():
    shutil.rmtree(test_project_path)

print("=" * 60)
print("1️⃣  TESTING: Project.init()")
print("=" * 60)

# Initialize new project
project = Project.init(
    name="test_sdk",
    path=test_project_path
)

print(f"\n✅ Project created: {project}")
print(f"   Name: {project.name}")
print(f"   Path: {project.path}")
print(f"   Version: {project.version}")

# ============================================================
# 2. Load Existing Project
# ============================================================

print("\n" + "=" * 60)
print("2️⃣  TESTING: Project.load()")
print("=" * 60)

project = Project.load(test_project_path)
print(f"✅ Project loaded: {project}")

# ============================================================
# 3. Project Configuration
# ============================================================

print("\n" + "=" * 60)
print("3️⃣  TESTING: Config Operations")
print("=" * 60)

# Get config value
device = project.get_config("defaults.device")
batch_size = project.get_config("defaults.batch_size")
print(f"Device: {device}")
print(f"Batch size: {batch_size}")

# Set config value
project.set_config("defaults.batch_size", 32)
project.save_config()
print(f"✅ Updated batch_size to 32")

# ============================================================
# 4. Import Dataset
# ============================================================

print("\n" + "=" * 60)
print("4️⃣  TESTING: Import Dataset")
print("=" * 60)

# Create sample images directory
sample_images_dir = Path("./sample_images")
sample_images_dir.mkdir(exist_ok=True)

# Create some dummy image files for testing
from PIL import Image
import numpy as np

for i in range(5):
    img = Image.fromarray(np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8))
    img.save(sample_images_dir / f"test_image_{i}.jpg")

print(f"Created {len(list(sample_images_dir.glob('*.jpg')))} test images")

# Import dataset
try:
    dataset = project.import_dataset(
        source=sample_images_dir,
        name="test_dataset",
        classes=["cat", "dog", "bird"],
        recursive=False,
        copy=True,
        validate=True
    )
    print(f"✅ Dataset imported: {dataset}")
except Exception as e:
    print(f"⚠️  Import may have failed (expected if service not fully implemented): {e}")

# ============================================================
# 5. List Datasets
# ============================================================

print("\n" + "=" * 60)
print("5️⃣  TESTING: List Datasets")
print("=" * 60)

datasets = project.list_datasets()
print(f"Found {len(datasets)} dataset(s):")
for ds in datasets:
    print(f"  - {ds.name}: {ds.images} images, status={ds.status}")

# ============================================================
# 6. Dataset Operations
# ============================================================

if datasets:
    print("\n" + "=" * 60)
    print("6️⃣  TESTING: Dataset Operations")
    print("=" * 60)

    dataset = datasets[0]

    # Get dataset info
    info = dataset.info()
    print(f"\n📊 Dataset Info:")
    print(f"   Name: {info.name}")
    print(f"   Images: {info.total_images}")
    print(f"   Classes: {info.classes}")
    print(f"   Size: {info.size}")
    print(f"   Status: {info.status}")

    # Get split counts
    splits = dataset.get_split_counts()
    print(f"\n📁 Split Counts:")
    for split, count in splits.items():
        print(f"   {split}: {count}")

    # List classes
    classes = dataset.list_classes()
    print(f"\n🏷️  Classes: {classes}")

    # Test class operations
    try:
        print(f"\n➕ Testing add_class...")
        new_id = dataset.add_class("fish")
        print(f"   Added 'fish' with ID: {new_id}")
        print(f"   Classes now: {dataset.list_classes()}")
    except Exception as e:
        print(f"   ⚠️  {e}")

    # List images
    try:
        print(f"\n🖼️  Testing list_images...")
        images, total = dataset.list_images(limit=10)
        print(f"   Found {total} total images")
        print(f"   Showing {len(images)} images")
        for img in images[:3]:
            print(f"      - {img.get('filename', 'unknown')}")
    except Exception as e:
        print(f"   ⚠️  {e}")

# ============================================================
# 7. Annotation Operations
# ============================================================

if datasets:
    print("\n" + "=" * 60)
    print("7️⃣  TESTING: Annotation Operations")
    print("=" * 60)

    from modelcub import Box

    dataset = datasets[0]
    test_image_id = "test_image_0"

    # Create sample boxes
    boxes = [
        Box(class_id=0, x=0.5, y=0.5, w=0.2, h=0.2),
        Box(class_id=1, x=0.3, y=0.3, w=0.15, h=0.15),
    ]

    try:
        print(f"\n💾 Testing save_annotation...")
        result = dataset.save_annotation(test_image_id, boxes)
        print(f"   Saved {len(boxes)} boxes for {test_image_id}")
        print(f"   Result: {result}")
    except Exception as e:
        print(f"   ⚠️  {e}")

    try:
        print(f"\n📖 Testing get_annotation...")
        annotation = dataset.get_annotation(test_image_id)
        print(f"   Retrieved annotation: {annotation}")
    except Exception as e:
        print(f"   ⚠️  {e}")

    try:
        print(f"\n📊 Testing annotation_stats...")
        stats = dataset.annotation_stats()
        print(f"   Stats: {stats}")
    except Exception as e:
        print(f"   ⚠️  {e}")

# ============================================================
# 8. Job Operations
# ============================================================

print("\n" + "=" * 60)
print("8️⃣  TESTING: Job Operations")
print("=" * 60)

try:
    job_manager = project.jobs
    print(f"✅ JobManager loaded: {job_manager}")

    # List jobs
    jobs = job_manager.list_jobs()
    print(f"   Found {len(jobs)} job(s)")

    # Create a test job (if dataset exists)
    if datasets:
        dataset = datasets[0]
        print(f"\n🔧 Testing create_job...")
        job = job_manager.create_job(
            dataset_name=dataset.name,
            config={"batch_size": 16}
        )
        print(f"   Created job: {job}")
        print(f"   Job ID: {job.id}")
        print(f"   Status: {job.status}")

except Exception as e:
    print(f"⚠️  Job operations not fully implemented: {e}")

# ============================================================
# 9. Context Manager
# ============================================================

print("\n" + "=" * 60)
print("9️⃣  TESTING: Context Manager")
print("=" * 60)

with Project.load(test_project_path) as p:
    print(f"✅ Using project in context: {p.name}")
    p.set_config("defaults.batch_size", 64)
    print(f"   Updated batch_size to 64 (auto-saves on exit)")

# Verify the change persisted
project = Project.load(test_project_path)
print(f"✅ Config persisted: batch_size = {project.get_config('defaults.batch_size')}")

# ============================================================
# 10. Cleanup (Optional)
# ============================================================

print("\n" + "=" * 60)
print("🧹 CLEANUP")
print("=" * 60)

response = input("\nDelete test project? (y/n): ")
if response.lower() == 'y':
    try:
        project.delete(confirm=True)
        print("✅ Project deleted")
    except Exception as e:
        print(f"⚠️  {e}")

    # Clean up sample images
    if sample_images_dir.exists():
        shutil.rmtree(sample_images_dir)
        print("✅ Sample images cleaned up")
else:
    print(f"Project kept at: {test_project_path}")

print("\n" + "=" * 60)
print("✅ SDK PLAYGROUND COMPLETE")
print("=" * 60)

✅ Logging enabled

1️⃣  TESTING: Project.init()
2025-10-18 17:11:42,938 - modelcub.services - INFO - [init_project] Starting...
2025-10-18 17:11:42,944 - modelcub.services - INFO - [init_project] Success (5.60ms)

✅ Project created: ModelCub Project: test_sdk
   Name: test_sdk
   Path: /Users/malikmacbook/Desktop/ModelCub/exemple/test_sdk_project
   Version: 1.0.0

2️⃣  TESTING: Project.load()
✅ Project loaded: ModelCub Project: test_sdk

3️⃣  TESTING: Config Operations
Device: mps
Batch size: 16
✅ Updated batch_size to 32

4️⃣  TESTING: Import Dataset
Created 5 test images
✅ Dataset imported: Dataset: test_dataset (5 images)

5️⃣  TESTING: List Datasets
Found 1 dataset(s):
  - test_dataset: 5 images, status=unlabeled

6️⃣  TESTING: Dataset Operations

📊 Dataset Info:
   Name: test_dataset
   Images: 5
   Classes: ['cat', 'dog', 'bird']
   Size: 1.2 MB
   Status: unlabeled

📁 Split Counts:
   train: 0
   val: 0
   test: 0
   unlabeled: 5

🏷️  Classes: ['cat', 'dog', 'bird']

➕ Testing 